In [28]:
import numpy as np
import cv2 as cv
import sys
import pandas as pd
import os
import cv2 as cv
import mediapipe as mp

In [53]:
def construct_yolo_v3():
    f = open('coco_names.txt', 'r')
    class_names = [line.strip() for line in f.readlines()]

    model = cv.dnn.readNet('yolov3.weights', 'yolov3.cfg')
    layer_names = model.getLayerNames()
    out_layers = [layer_names[i - 1] for i in model.getUnconnectedOutLayers()]
    
    return model, out_layers, class_names

def yolo_detect(img, yolo_model, out_layers):
    height, width = img.shape[0], img.shape[1]
    test_img = cv.dnn.blobFromImage(img, 1.0 / 256, (448, 448), (0, 0, 0), swapRB=True)
    
    yolo_model.setInput(test_img)
    output3 = yolo_model.forward(out_layers)
    
    box, conf, id = [], [], []  # 박스, 신뢰도, 부류 번호
    for output in output3:
        for vec85 in output:
            scores = vec85[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # 신뢰도가 50% 이상인 경우만 취함
                centerx, centery = int(vec85[0] * width), int(vec85[1] * height)
                w, h = int(vec85[2] * width), int(vec85[3] * height)
                x, y = int(centerx - w / 2), int(centery - h / 2)
                box.append([x, y, x + w, y + h])
                conf.append(float(confidence))
                id.append(class_id)
            
    ind = cv.dnn.NMSBoxes(box, conf, 0.5, 0.4)
    objects = [box[i] + [conf[i]] + [id[i]] for i in range(len(box)) if i in ind]
    return objects

def sort_to_csv(video_path):

    model, out_layers, class_names = construct_yolo_v3()
    colors = np.random.uniform(0, 255, size=(100, 3))


    from sort import Sort
    sort = Sort()
    cap = cv.VideoCapture(video_path)

    if not cap.isOpened(): 
        sys.exit('동영상 파일을 열 수 없습니다.')

    # 결과를 저장할 리스트 생성
    tracking_results = []

    frame_number = 0
    while True:
        ret, frame = cap.read()
        if not ret: 
            break
        
        res = yolo_detect(frame, model, out_layers)   
        persons = [res[i] for i in range(len(res)) if res[i][5] == 0]

        if len(persons) == 0: 
            tracks = sort.update()
        else:
            tracks = sort.update(np.array(persons))
        
        for track in tracks:
            x1, y1, x2, y2, track_id = track.astype(int)
            # 리스트에 결과 추가
            tracking_results.append({'Frame': frame_number, 'ID': track_id, 'Box': [x1, y1, x2, y2]})

        frame_number += 1

        key = cv.waitKey(1) 
        if key == ord('q'): 
            break 

    cap.release()
    cv.destroyAllWindows()
    
    tracking_results_df = pd.DataFrame(tracking_results).reset_index(drop=True)

    return tracking_results_df

In [54]:
def extract_and_create_videos(video_path, tracking_data, output_folder):
    cap = cv.VideoCapture(video_path)
    if not cap.isOpened():
        raise Exception("Error opening video file")

    # Get video properties
    frame_rate = cap.get(cv.CAP_PROP_FPS)
    video_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    video_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

    # Create a dictionary to store video writers for each ID
    video_writers = {}

    # Process each entry in the tracking data
    for index, row in tracking_data.iterrows():
        frame_number = row['Frame']
        box = eval(row['Box'])
        person_id = row['ID']
        x1, y1, x2, y2 = box

        # Ensure coordinates are within bounds
        x1, y1, x2, y2 = max(x1, 0), max(y1, 0), min(x2, video_width), min(y2, video_height)

        # Set the video to the specific frame
        cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = cap.read()
        if not ret or frame is None:
            continue

        # Crop and resize frame
        cropped_frame = frame[y1:y2, x1:x2]
        if cropped_frame.size == 0:
            continue  # Skip if the cropped frame is empty
        resized_frame = cv.resize(cropped_frame, (300, 300))

        # Check if the video writer for this ID exists
        if person_id not in video_writers:
            output_filename = os.path.join(output_folder, f"video_id_{person_id}.mp4")
            fourcc = cv.VideoWriter_fourcc(*'mp4v')
            video_writers[person_id] = cv.VideoWriter(output_filename, fourcc, frame_rate, (300, 300))

        # Write the frame to the video
        video_writers[person_id].write(resized_frame)

    # Release all video writers
    for writer in video_writers.values():
        writer.release()

    cap.release()

In [ ]:
def video_square(df,video_path):

    tracking_data = {}

    for _, row in df.iterrows():
        frame_number = row['Frame']
        box_str = str(row['Box'])
        box = eval(row['Box'])
        track_id = row['ID']
        if frame_number not in tracking_data:
            tracking_data[frame_number] = []
        tracking_data[frame_number].append((track_id, box))

    cap = cv.VideoCapture(video_path)
    
    if not cap.isOpened():
        sys.exit('동영상 파일을 열 수 없습니다.')

    # Define the codec and create VideoWriter object
    #mode = 'avi'
    mode = 'mp4'
    if mode == 'avi':
        fourcc = cv.VideoWriter_fourcc(*'XVID')
        out = cv.VideoWriter('output.avi', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))
    elif mode == 'mp4':
        fourcc = cv.VideoWriter_fourcc(*'mp4v')
        out = cv.VideoWriter('output2.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    frame_number = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_number in tracking_data:
            for track_id, box in tracking_data[frame_number]:
                x1, y1, x2, y2 = box
                cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv.putText(frame, str(track_id), (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)


        out.write(frame)

        cv.imshow('Tracking', frame)
        frame_number += 1

        if cv.waitKey(1) & 0xFF == ord('q'):
            break

    # Release everything if job is finished
    cap.release()
    out.release()
    cv.destroyAllWindows()

In [19]:
def skeleton(videopath):

    mp_pose=mp.solutions.pose
    mp_drawing=mp.solutions.drawing_utils
    mp_styles=mp.solutions.drawing_styles

    pose=mp_pose.Pose(static_image_mode=False,enable_segmentation=True,min_detection_confidence=0.5,min_tracking_confidence=0.5)

    #cap=cv.VideoCapture(0)
    cap=cv.VideoCapture(videopath)

    while True:
        ret,frame=cap.read()
        if not ret:
            print('프레임 획득에 실패하여 루프를 나갑니다.')
            break
        
        res=pose.process(cv.cvtColor(frame,cv.COLOR_BGR2RGB))
        
        mp_drawing.draw_landmarks(frame,res.pose_landmarks,mp_pose.POSE_CONNECTIONS,landmark_drawing_spec=mp_styles.get_default_pose_landmarks_style())
        
        cv.imshow('MediaPipe pose',cv.flip(frame,1)) # 좌우반전
        if cv.waitKey(5)==ord('q'):
            mp_drawing.plot_landmarks(res.pose_world_landmarks,mp_pose.POSE_CONNECTIONS)
            break

    cap.release()
    cv.destroyAllWindows()

In [24]:
video_path = '../data/mevid.mp4'
output_folder = '../OUTPUT'
outputvideo = '../OUTPUT/video_id_6.mp4'

In [56]:
csv_df = sort_to_csv(video_path)

In [29]:
csv_df.to_csv('../OUTPUT/sort_csv')

NameError: name 'csv_df' is not defined

In [ ]:
extract_and_create_videos(video_path, csv_df, output_folder)

In [14]:
tracking_data = video_square(csv_df,video_path)

: 

In [27]:
skeleton(outputvideo)

I0000 00:00:1700701405.579676       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


프레임 획득에 실패하여 루프를 나갑니다.
